In [2]:
import os
import re
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.common.by import By
from datetime import datetime
from openpyxl import load_workbook
import pandas as pd
import pickle
import xlsxwriter
import warnings
import shutil

warnings.filterwarnings("ignore", category=DeprecationWarning) 

class ReportScraper:
    def __init__(self, base_url):
        self.base_url = base_url
        self.report_name = []
        self.hhs_agencies = []
        self.issued_dates = []
        self.report_dict = {}
        self.driver = None
        self.report_links = []
        self.CR = []
        self.RIB=[]
        self.pdf_path = None


    def initialize_driver(self):
        options = Options()
        options.add_argument("--ignore-certificate-errors")
        self.driver = webdriver.Chrome(options=options)

    def is_url_reachable(self, url):
        try:
            response = requests.get(url)
            return response.status_code == 200
        except requests.exceptions.RequestException:
            return False
        

    def move_to_legacy(self, old_path,old_date, folder_type, pdf_filename, is_complete_report):
        
        if folder_type=='complete_reports':
            shutil.move(old_path, legacy_cr)
        else:
            shutil.move(old_path, legacy_rib)
        

      

    def extract_and_download_pdf(self, link,report, pdf_filename,date, pdf_path, is_complete_report):
        folder_type = 'complete_reports' if is_complete_report else 'report_in_brief'
        excel_filename="report_page_1.xlsx"
        if os.path.isfile(excel_filename):
            data=pd.read_excel(excel_filename)
            location = data.index[data.eq(report).any(axis=1)].tolist()
            if not os.path.exists(pdf_path):  # Check if the file exists
                self.download_pdf(link, pdf_path, pdf_filename, is_complete_report)
            else:
                for i in location:
                    if data["Issued Date"][i]!=date:
                        if is_complete_report:
                            self.move_to_legacy(data['complete report'][i],["Issued Date"][i], folder_type, pdf_filename, is_complete_report)
                            data['complete report'][i]=pdf_path
                            self.download_pdf(link, pdf_path, pdf_filename,is_complete_report)
                        else:
                            self.move_to_legacy(data['report in brief'][i],["Issued Date"][i], folder_type, pdf_filename, is_complete_report)
                            data['report in brief'][i]=pdf_path
                            self.download_pdf(link, pdf_path, pdf_filename,is_complete_report)
                        data["Issued Date"][i]==date
        else:
            if not os.path.exists(pdf_path):
                self.download_pdf(link, pdf_path, pdf_filename, is_complete_report)
                        

    def download_pdf(self, url, pdf_path, filename, is_complete_report):
        response = requests.get(url)
        with open(pdf_path, 'wb') as pdf_file:
            pdf_file.write(response.content)

         
        
    def scrape_report(self, report_links,issued_dates,report_name):
        if len(report_links)==len(issued_dates):
            for i in range(len(report_links)):
                    report = report_links[i]
                    date=issued_dates[i]
                    reportname=report_name[i]
                    if report!= 'N/A':
                        if report.endswith(".asp"):
                            self.extract_pdf_links(report,date,reportname)
                        else:
                            if report.startswith("oig."):
                                report_url = "http://oig.hhs.gov" + report
                                if self.is_url_reachable(report_url):
                                    pdf_filename = os.path.basename(report_url)
                                    pdf_path = os.path.join(current_cr, pdf_filename)
                                    path=os.path.abspath(pdf_path)
                                    self.CR.append(os.path.abspath(pdf_path))
                                    self.extract_and_download_pdf(report_url,reportname, pdf_filename,date, path, is_complete_report=True)
                                    self.RIB.append('N/A')
                                else:
                                    self.CR.append('N/A')
                                    self.RIB.append('N/A')
                            else:
                                if self.is_url_reachable(report):
                                    pdf_filename = os.path.basename(report)
                                    pdf_path = os.path.join(current_cr, pdf_filename)
                                    path=os.path.abspath(pdf_path)
                                    self.CR.append(os.path.abspath(pdf_path))
                                    self.extract_and_download_pdf(report,reportname, pdf_filename,date, path, is_complete_report=True)
                                    self.RIB.append('N/A')

                                elif report.startswith("http://oig.hh."):
                                    report=report[:13]+"s"+report[13:]
                                    report=report[:4]+"s"+report[4:] 
                                    pdf_path = os.path.join(current_cr, pdf_filename)
                                    path=os.path.abspath(pdf_path)
                                    self.CR.append(os.path.abspath(pdf_path))
                                    self.extract_and_download_pdf(report,reportname,pdf_filename,date, path, is_complete_report=True)
                                    self.RIB.append('N/A')

                                elif report.startswith("http:/"):
                                    report=report[:4]+"s"+report[4:]
                                    pdf_path = os.path.join(current_cr, pdf_filename)
                                    path=os.path.abspath(pdf_path)
                                    self.CR.append(os.path.abspath(pdf_path))
                                    self.extract_and_download_pdf(report,reportname, pdf_filename,date,path,  is_complete_report=True)
                                    self.RIB.append('N/A')
                                else:
                                    self.CR.append('N/A')
                                    self.RIB.append('N/A')
                    else:
                        self.CR.append('Report pending')
                        self.RIB.append('Report pending')
                

    def extract_pdf_links(self, report_url,date,reportname):
        self.driver.get(report_url)
        time.sleep(5)  # Wait for the page to load (adjust as needed)
        if self.is_url_reachable(report_url):
            soup1 = BeautifulSoup(self.driver.page_source, "html.parser")
            a_elements = soup1.find('p', class_='report-metadata').find_all('a')
            href_links = [a['href'] for a in a_elements]
            if len(href_links) == 2:
                for i in href_links:
                    link = "https://oig.hhs.gov" + i
                    if link.endswith("RIB.pdf"):
                        if self.is_url_reachable(link):
                            pdf_filename = os.path.basename(link)
                            pdf_path = os.path.join(current_rib, pdf_filename)
                            path=os.path.abspath(pdf_path)
                            self.RIB.append(os.path.abspath(pdf_path))
                            self.extract_and_download_pdf(link, reportname,pdf_filename,date,path,  is_complete_report=False)
                        
                        elif report.startswith("http://oig.hh."):
                            report=report[:13]+"s"+report[13:]
                            report=report[:4]+"s"+report[4:] 
                            pdf_path = os.path.join(current_cr, pdf_filename)
                            path=os.path.abspath(pdf_path)
                            self.RIB.append(os.path.abspath(pdf_path))
                            self.extract_and_download_pdf(report,reportname,pdf_filename,date, path, is_complete_report=False)
                            
                        elif report.startswith("http:/"):
                            report=report[:4]+"s"+report[4:]
                            pdf_path = os.path.join(current_cr, pdf_filename)
                            path=os.path.abspath(pdf_path)
                            self.RIB.append(os.path.abspath(pdf_path))
                            self.extract_and_download_pdf(report,reportname, pdf_filename,date, path,is_complete_report=False)
                            
                        else:
                            self.RIB.append('N/A')
                    else:
                        if self.is_url_reachable(link):
                            pdf_filename = os.path.basename(link)
                            pdf_path = os.path.join(current_cr, pdf_filename)
                            path=os.path.abspath(pdf_path)
                            self.CR.append(os.path.abspath(pdf_path))
                            self.extract_and_download_pdf(link,reportname, pdf_filename,date,path, is_complete_report=True)
                        
                        elif report.startswith("http://oig.hh."):
                            report=report[:13]+"s"+report[13:]
                            report=report[:4]+"s"+report[4:] 
                            pdf_path = os.path.join(current_cr, pdf_filename)
                            path=os.path.abspath(pdf_path)
                            self.CR.append(os.path.abspath(pdf_path))
                            self.extract_and_download_pdf(report,reportname, pdf_filename,date, path,is_complete_report=True)  

                        elif report.startswith("http:/"):
                            report=report[:4]+"s"+report[4:]
                            pdf_path = os.path.join(current_cr, pdf_filename)
                            path=os.path.abspath(pdf_path)
                            self.CR.append(os.path.abspath(pdf_path))
                            self.extract_and_download_pdf(report, reportname,pdf_filename, date,path,is_complete_report=True)
                            
                        else:
                            self.CR.append('N/A')
            else:
                for i in href_links:
                    link = "https://oig.hhs.gov" + i
                    if self.is_url_reachable(link):
                        pdf_filename = os.path.basename(link)
                        pdf_path = os.path.join(current_cr, pdf_filename)
                        path=os.path.abspath(pdf_path)
                        self.CR.append(os.path.abspath(pdf_path))
                        self.extract_and_download_pdf(link,reportname, pdf_filename,date,path,is_complete_report=True)
                        self.RIB.append("N/A")

                    elif report.startswith("http://oig.hh."):
                            report=report[:13]+"s"+report[13:]
                            report=report[:4]+"s"+report[4:] 
                            pdf_path = os.path.join(current_cr, pdf_filename)
                            path=os.path.abspath(pdf_path)
                            self.CR.append(os.path.abspath(pdf_path))
                            self.extract_and_download_pdf(report,reportname, pdf_filename,date,path, is_complete_report=True)
                            self.RIB.append('N/A')

                    elif report.startswith("http:/"):
                        report=report[:4]+"s"+report[4:]
                        pdf_path = os.path.join(current_cr, pdf_filename)
                        path=os.path.abspath(pdf_path)
                        self.CR.append(os.path.abspath(pdf_path))
                        self.extract_and_download_pdf(report,reportname, pdf_filename,date, path, is_complete_report=True)
                        self.RIB.append('N/A')
                    else:
                        self.CR.append('N/A')
                        self.RIB.append("N/A")
        else:
            self.CR.append("N/A")
            self.RIB.append("N/A")

    def create_report_dict(self, report_name, hhs_agencies, issued_dates, CR, RIB):
        
        for i in range(len(report_name)):
            self.report_dict[report_name[i]] = {
                "HHS Agency": hhs_agencies[i],
                "Issued Date": issued_dates[i],
                "complete report": CR[i],
                "report in brief":RIB[i]
            }
        return self.report_dict


    def create_metadata_file(self, excel_filename,report_dict):
        if not os.path.isfile(excel_filename):
            report_df = pd.DataFrame.from_dict(self.report_dict, orient='index')
            report_df= report_df.reset_index()
            report_df = report_df.rename(columns={'index': 'Report Name'})
            with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
                report_df.to_excel(writer, sheet_name='Sheet1', index=False)
        else:
            os.remove(excel_filename)
            report_df = pd.DataFrame.from_dict(self.report_dict, orient='index')
            report_df= report_df.reset_index()
            report_df = report_df.rename(columns={'index': 'Report Name'})
            with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
                report_df.to_excel(writer, sheet_name='Sheet1', index=False)
            

    def run_scraper(self, num_pages, excel_filename):
        
        self.initialize_driver()

        for page_number in range(1, num_pages + 1):
            page_url = self.base_url + str(page_number)
            print(page_url)
            self.driver.get(page_url)
            time.sleep(5)
            soup = BeautifulSoup(self.driver.page_source, "html.parser")
            report_elements_xpath = "/html//section[@id='results']/div[@class='grid-container']/div[2]/div[@class='grid-col-fill']/ul[1]/li/div[@class='usa-card__container']"
            report_elements = self.driver.find_elements(By.XPATH, report_elements_xpath)

            for report_element in report_elements:
                title_element = report_element.find_element(By.XPATH, ".//header[@class='usa-card__header']/h2")
                self.report_name.append(title_element.text.strip())

            dl_elements = soup.find_all('dl', class_='pep-metadata--inline')
            for ele in dl_elements:
                dt_elements = ele.find_all('dt', class_='pep-metadata__term')
                dd_elements = ele.find_all('dd', class_='pep-metadata__def')
                elements = ele.find_all('a', class_='usa-link')

                hhs_agency = dd_elements[1].text.strip() if len(dd_elements) > 1 else 'N/A'
                issued_date = dd_elements[2].text.strip() if len(dd_elements) > 2 else 'N/A'

                self.hhs_agencies.append(hhs_agency)
                self.issued_dates.append(issued_date)

            dl_elements = soup.find_all('dl', class_='pep-metadata--inline')
            for ele in dl_elements:
                elements = ele.find_all('a', class_='usa-link')
                report_link = elements[0]['href'].strip() if elements else 'N/A'
                self.report_links.append(report_link)

            self.scrape_report(self.report_links,self.issued_dates,self.report_name)

            self.report_links = []

        report_dict=self.create_report_dict(self.report_name, self.hhs_agencies, self.issued_dates, self.CR, self.RIB)
        
        self.create_metadata_file(excel_file,report_dict)

        self.driver.quit()




current_cr = 'current/complete_reports'
current_rib='current/report_in_brief'
legacy_cr='legacy/complete_reports'
legacy_rib='legacy/report_in_brief'
if not os.path.exists(current_cr):
    os.makedirs(current_cr)
if not os.path.exists(current_rib):
    os.makedirs(current_rib)
if not os.path.exists(legacy_cr):
    os.makedirs(legacy_cr)
if not os.path.exists(legacy_rib):
    os.makedirs(legacy_rib)


base_url = "https://oig.hhs.gov/reports-and-publications/recommendations/tracker/?page="

excel_file = 'report_page_1.xlsx'
                
scraper = ReportScraper(base_url)

# Run scraper
scraper.run_scraper(num_pages=1,excel_filename=excel_file)

https://oig.hhs.gov/reports-and-publications/recommendations/tracker/?page=1


In [10]:
! pip  install  xlsxwriter


   ---------------------------------------- 0.0/154.8 kB ? eta -:--:--
   ------- ------------------------------- 30.7/154.8 kB 640.0 kB/s eta 0:00:01
   -------------------- ------------------ 81.9/154.8 kB 919.0 kB/s eta 0:00:01
   ------------------------------- -------- 122.9/154.8 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- 154.8/154.8 kB 768.7 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
